### IE801(B) Logistics Management HW3

- Lee Kanghoon, 20203421

In [40]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import copy

import gurobipy as gp
from gurobipy import GRB

### Branch-and-Price algorithm

Implement the Branch-and-Price algorithm for solving the Constrained Shortest Path Problem (CSPP)

![nn](./Images/HW3_image1.png)

reference : https://en.wikipedia.org/wiki/Branch_and_price

### Problem 1.

Using your Branch-and-Price code, solve the toy example considered in the class. 
![nn](./Images/HW3_image2.png)

reference : Desrosiers, Jacques, and Marco E. Lübbecke. "A primer in column generation." Column generation. Springer, Boston, MA, 2005. 1-32.

### Practice : solve the constrained shortest path problem with MIP

![nn](./Images/HW3_image3.png)

In [119]:
def solve_CSPP_example():
    
    # problem parameter
    num_nodes = 6
    num_edges = 10
    edges = [(1, 2, {'cost': 1, 'time': 10}),
            (1, 3, {'cost': 10, 'time': 3}),
            (2, 4, {'cost': 1, 'time': 1}),
            (2, 5, {'cost': 2, 'time': 3}),
            (3, 2, {'cost': 1, 'time': 2}),
            (3, 4, {'cost': 5, 'time': 7}),
            (3, 5, {'cost': 12, 'time': 3}),
            (4, 5, {'cost': 10, 'time': 1}),
            (4, 6, {'cost': 1, 'time': 7}),
            (5, 6, {'cost': 2, 'time': 2})]

    G = nx.DiGraph()
    G.add_node(num_nodes)
    G.add_edges_from(edges)

    model = gp.Model()
    model.Params.LogToConsole = 0
    variables = gp.tupledict()

    # (1.1), (1.6)
    for edge in edges:
        i, j, data = edge
        variables[i, j] = model.addVar(obj=data['cost'],
                                       vtype=GRB.BINARY)

    # (1.2) ~ (1.4)
    for n in range(1, num_nodes+1):
        term1 = sum(variables[i, j] for i, j in G.out_edges(n))
        term2 = sum(variables[i, j] for i, j in G.in_edges(n))
        equation = term1 - term2

        if n == 1:
            model.addConstr(equation == 1)
        elif n == num_nodes:
            model.addConstr(equation == -1)
        else:
            model.addConstr(equation == 0)

    # (1.5)
    equation = None
    for edge in edges:
        i, j, data = edge
        equation += data['time'] * variables[i, j]
    model.addConstr(equation <= 14)

    # optimize!
    model._vars = variables
    model.optimize()

    solution = model.getAttr('x', variables)
    value = model.objVal
    
    G = nx.DiGraph()
    G.add_node(num_nodes)

    for i, j in solution:
        if solution[(i, j)] == 1.0:
            G.add_edge(i, j)
    path = list(nx.algorithms.simple_paths.shortest_simple_paths(G, 1, 6))[0]
    print('path  : ', path)
    print('value : ', value)

In [120]:
solve_CSPP_example()

path  :  [1, 3, 2, 4, 6]
value :  13.0


### master problem

![nn](./Images/HW3_image4.png)

In [198]:
def BP_CSSP(num_nodes, edges):

    path = {}
    path_list = []
    cost_list = []
    time_list = []
    cnt = 0
    
    for i in range(6):
        print("----------")
        print("iteration.. : ", i+1)
        cnt += 1
        num_paths = len(path_list)
        print("cost : ", cost_list)
        print("time : ", time_list)

        model = gp.Model()
        model.Params.LogToConsole = 0
        variables = gp.tupledict()

        for i in range(num_paths):
            variables[i] = model.addVar(obj=cost_list[i])
        variables['y'] = model.addVar(obj=10000)

        const_r = model.addConstr(sum(time_list[i] * variables[i] for i in range(num_paths)) <= 14)
        const_c = model.addConstr(sum(variables[i] for i in range(num_paths)) + variables['y'] == 1)

        model._vars = variables
        model.optimize()

        solution = model.getAttr('x', variables)
        value = model.objVal
        
        print(solution, value)

        pi1 = const_r.Pi
        pi0 = const_c.Pi
        
        for edge in edges:
            edge[-1]['revised_cost'] = edge[-1]['cost'] - pi1 * edge[-1]['time']
        
        G = nx.DiGraph()
        G.add_node(num_nodes)
        G.add_edges_from(edges)

        length, path_sub = nx.algorithms.shortest_paths.weighted.single_source_bellman_ford(G, 1, 6, 'revised_cost')
        min_reduced_cost = length - pi0
        print(min_reduced_cost)
        print(length, path_sub)
        cost_sub = nx.classes.path_weight(G, path_sub, 'cost')
        time_sub = nx.classes.path_weight(G, path_sub, 'time')
        
        path[cnt] = path_sub
        path_list.append(cnt)
        cost_list.append(cost_sub)
        time_list.append(time_sub)

In [199]:
# problem parameter
num_nodes = 6
edges = [(1, 2, {'cost': 1, 'time': 10}),
        (1, 3, {'cost': 10, 'time': 3}),
        (2, 4, {'cost': 1, 'time': 1}),
        (2, 5, {'cost': 2, 'time': 3}),
        (3, 2, {'cost': 1, 'time': 2}),
        (3, 4, {'cost': 5, 'time': 7}),
        (3, 5, {'cost': 12, 'time': 3}),
        (4, 5, {'cost': 10, 'time': 1}),
        (4, 6, {'cost': 1, 'time': 7}),
        (5, 6, {'cost': 2, 'time': 2})]

BP_CSSP(num_nodes, edges)

----------
iteration.. :  1
cost :  []
time :  []
{'y': 1.0} 10000.0
-9997.0
3.0 [1, 2, 4, 6]
----------
iteration.. :  2
cost :  [3]
time :  [18]
{0: 0.7777777777777778, 'y': 0.22222222222222232} 2224.5555555555566
-5532.888888888889
4467.111111111111 [1, 3, 5, 6]
----------
iteration.. :  3
cost :  [3, 24]
time :  [18, 8]
{0: 0.6, 1: 0.4, 'y': 0.0} 11.400000000000002
-4.799999999999997
36.0 [1, 3, 2, 5, 6]
----------
iteration.. :  4
cost :  [3, 24, 15]
time :  [18, 8, 10]
{0: 0.5000000000000001, 1: 0.0, 2: 0.49999999999999983, 'y': 0.0} 8.999999999999998
-2.5
27.5 [1, 2, 5, 6]
----------
iteration.. :  5
cost :  [3, 24, 15, 5]
time :  [18, 8, 10, 15]
{0: 0.0, 1: 0.0, 2: 0.19999999999999993, 3: 0.8, 'y': 0.0} 6.999999999999999
0.0
35.0 [1, 2, 5, 6]
----------
iteration.. :  6
cost :  [3, 24, 15, 5, 5]
time :  [18, 8, 10, 15, 15]
{0: 0.0, 1: 0.0, 2: 0.19999999999999993, 3: 0.0, 4: 0.8, 'y': 0.0} 6.999999999999999
0.0
35.0 [1, 2, 5, 6]
